# 0 Import

In [95]:
import numpy as np 
np.set_printoptions(threshold=10000,suppress=True) 
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string
warnings.filterwarnings('ignore')

# 1. Data Analyse

In [ ]:
data=pd.read_csv('PubMed-multi-label-dataset.csv',sep=',')
print(len(data))
data.head(3)

In [ ]:
labels = data.iloc[:, -14:]
print("count of number of 1 per Label")
labels.where(labels == 1).count()

In [ ]:
plt.matshow(labels.corr())

# 2. Modelisation

Pour modéliser notre problème d'apprentissage suppervisé, on défini notre plan d'attaque.

### Entrées

- abstractText 
- Potentiellement Title & meshMajor

### Sorties

- Les 14 labels

### Evaluations

- F1-score en raison du déséquilibre de la donnée.

### Approche

- Oversampling ou undersampling

# 3. Clean Data

In [ ]:
import gensim
corpus = data['abstractText'].astype(str) + data['Title'].astype(str)
corpus

In [ ]:
corpus = corpus.apply(remove_stopwords)
corpus = corpus.apply(preprocess_string)
corpus = corpus.apply(lambda x : ' '.join(x))
corpus

In [ ]:
title = data['Title'].astype(str)
title = title.apply(remove_stopwords)
title = title.apply(preprocess_string)
title = title.apply(lambda x : ' '.join(x))
title

In [129]:
#X_train, X_test, y_train, y_test = train_test_split(corpus, labels, test_size=0.5)

## Apprentissage du Word2vec

In [134]:
corpus_w2v = corpus.apply(lambda line : gensim.utils.simple_preprocess((line)))

In [ ]:
import multiprocessing
cores=multiprocessing.cpu_count()
cores

In [136]:
model_size=100
model=gensim.models.Word2Vec(corpus_w2v, vector_size=model_size, sg=0, window=5, min_count=2, workers=cores-1)

In [ ]:
for i in range(100):
    model.train(corpus_w2v, total_examples=len(corpus_w2v), epochs=1)
    print(i, end=' ')

In [ ]:
model.save('./models/Word2vec_entraine.h5')
len(model.wv.index_to_key)

In [ ]:
corpus

In [ ]:
tfidf = TfidfVectorizer()
tfidf.fit_transform(corpus)
tfidf_dict = {}
for ele1, ele2 in zip(tfidf.get_feature_names_out(), tfidf.idf_):
    tfidf_dict[ele1] = ele2
tfidf_dict

In [ ]:
model=gensim.models.Word2Vec.load('./models/Word2vec_entraine.h5')
def encode_sentence(sentence: str):
    vec = np.zeros((1, model.vector_size))
    for word in sentence.split(" "):
        try:
            if word in tfidf_dict.keys():
                vec += tfidf_dict[word] * model.wv[word]
            else:
                vec += model.wv[word]
        except:
            pass
    return vec

encode_sentence("paraffin embed tissu section patient")

In [ ]:
title2 = title.apply(encode_sentence)

In [ ]:
model.wv['easy']

In [ ]:
model.wv.most_similar(positive='easy',topn=10)

In [ ]:
model.wv.doesnt_match(['food','drink','play'])

In [ ]:
model.wv.most_similar(positive=['king','woman'],negative=['man'],topn=10)